In [1]:
%load_ext sql

In [2]:
import configparser
import sql_queries as sq

### Introduction
This notebook tests selected queries from `sql_queries.py` to:
1. Debug and tune the queries
1. Justify the chouce of distribution style and keys for the DWH tables

The code below caters for an active RedShift endpoint and IAM role for S3 access.
> Run `dwh_iac.ipynb` first to create the RedShift cluster and setup access!

In [3]:
dwh_cfg = configparser.ConfigParser()
dwh_cfg.read('dwh.cfg')

conn_values = (dwh_cfg.get('DB','DB_USER'),
               dwh_cfg.get('DB','DB_PASSWORD'),
               dwh_cfg.get('DB','DB_ENDPOINT'),
               dwh_cfg.get('DB','DB_PORT'),
               dwh_cfg.get('DB','DB_NAME'))

con = "postgresql://{}:{}@{}:{}/{}".format(*conn_values)
%sql $con

'Connected: dwhuser@dwh'

Drop all DWH tables if you like to start from scratch. `$con` is [supperssing the connection string](https://stackoverflow.com/q/53818698) from output.

In [ ]:
for query in sq.drop_table_queries:
    %sql $con $query

### Create and populate the staging tables

In [4]:
%sql $con $sq.staging_events_table_create
%sql $con $sq.staging_songs_table_create

Done.
Done.


[]

In [5]:
%%time
%sql $con $sq.staging_events_copy

Done.
CPU times: user 2.06 ms, sys: 8.21 ms, total: 10.3 ms
Wall time: 3.62 s


[]

In [6]:
%%time
%sql $con $sq.staging_songs_copy

Done.
CPU times: user 9.99 ms, sys: 369 µs, total: 10.4 ms
Wall time: 2min 45s


[]

With 2 nodes:

### Investigate the staging tables
How many fact records can we potentially have in table `songplays`?

In [7]:
%%sql $con 
SELECT COUNT (*) FROM (SELECT DISTINCT ts, userId FROM staging_events
WHERE   page    = 'NextSong'
    AND ts     is NOT NULL
    AND userId is NOT NULL
    AND level  is NOT NULL)

1 rows affected.


count
6820


Are there `songplays` for which no song can be matched from `staging_songs` by the artist name and song title?

In [8]:
%%sql $con 
SELECT COUNT (*) FROM (SELECT DISTINCT ts, userId FROM staging_events AS event
WHERE   page    = 'NextSong'
    AND ts     is NOT NULL
    AND userId is NOT NULL
    AND level  is NOT NULL
    AND NOT EXISTS (SELECT DISTINCT song_id FROM staging_songs AS song
                    WHERE event.song   = song.title
                      AND event.artist = song.artist_name
                   ))

1 rows affected.


count
6487


How many potential records in `time` dimension? The number of actual records will not exceed the number of facts in `songplays`.

In [9]:
%%sql $con
SELECT COUNT (*) FROM (SELECT DISTINCT ts FROM staging_events
WHERE   page    = 'NextSong'
    AND ts     is NOT NULL
    AND userId is NOT NULL
    AND level  is NOT NULL)

1 rows affected.


count
6813


So are there multiple `NextSong` events at the same time?

In [10]:
%%sql $con
SELECT ts, COUNT(userId) as users FROM staging_events
WHERE   page    = 'NextSong'
    AND ts     is NOT NULL
    AND userId is NOT NULL
    AND level  is NOT NULL
GROUP BY ts HAVING users > 1

7 rows affected.


ts,users
2018-11-14 04:53:36.796000,2
2018-11-15 18:55:04.796000,2
2018-11-24 14:29:47.796000,2
2018-11-23 14:41:51.796000,2
2018-11-28 16:36:15.796000,2
2018-11-28 19:59:23.796000,2
2018-11-27 17:28:50.796000,2


How many users are there in `users` dimension?

In [11]:
%%sql $con
SELECT COUNT (*) FROM (SELECT DISTINCT userId,  firstName,  lastName,  gender, level FROM staging_events
WHERE   userId is NOT NULL
    AND level  is NOT NULL)

1 rows affected.


count
105


For how many users at least one song can be matched from `staging_songs` by the artist name and song title??

In [12]:
%%sql $con 
SELECT COUNT (*) FROM (SELECT DISTINCT userId, firstName,  lastName,  gender, level FROM staging_events AS event
WHERE   page    = 'NextSong'
    AND ts     is NOT NULL
    AND userId is NOT NULL
    AND level  is NOT NULL
    AND EXISTS (SELECT DISTINCT song_id FROM staging_songs AS song
                WHERE event.song   = song.title
                    AND event.artist = song.artist_name
               ))

1 rows affected.


count
61


How many songs are there in `songs` dimension?

In [13]:
%%sql $con
SELECT COUNT (*) FROM (SELECT DISTINCT song_id FROM staging_songs
WHERE song_id is NOT NULL)

1 rows affected.


count
14896


How many of these songs will be mentioned in `songplays`?

In [14]:
%%sql $con
SELECT COUNT (*) FROM (SELECT DISTINCT song_id FROM staging_songs as song
WHERE song_id is NOT NULL
    AND EXISTS (SELECT DISTINCT ts FROM staging_events AS event
                WHERE   event.song   = song.title
                    AND event.artist = song.artist_name
                    AND event.page   = 'NextSong'
               ))

1 rows affected.


count
217


How many artists are there in `artists` dimension?

In [15]:
%%sql $con
SELECT COUNT (*) FROM (SELECT DISTINCT artist_id, artist_name FROM staging_songs
WHERE artist_id is NOT NULL
  AND song_id   is NOT NULL)

1 rows affected.


count
9993


How many of these artists will be mentioned in `songplays`?

In [16]:
%%sql $con
SELECT COUNT (*) FROM (SELECT DISTINCT artist_id FROM staging_songs as song
WHERE artist_id is NOT NULL
    AND EXISTS (SELECT DISTINCT ts FROM staging_events AS event
                WHERE   event.song   = song.title
                    AND event.artist = song.artist_name
                    AND event.page   = 'NextSong'
               ))

1 rows affected.


count
200


### Choose the distribution style
Based on the above counts we can choose the distribution style for fact and dimensions as:
- `songplays` and `time`: KEY with `start_time` as the distribution and the sort key
- `users`: ALL, sort key `user_id`
- `songs`: ALL, sort key `song_id`
- `artists`: ALL, sort key `artist_id`

### Create the star schema

In [17]:
%sql $con $sq.songplay_table_create
%sql $con $sq.user_table_create
%sql $con $sq.song_table_create
%sql $con $sq.artist_table_create
%sql $con $sq.time_table_create

Done.
Done.
Done.
Done.
Done.


[]

### Populate the star schema

In [18]:
%%time
%sql $con $sq.songplay_table_insert

333 rows affected.
CPU times: user 4.78 ms, sys: 10.8 ms, total: 15.6 ms
Wall time: 2.38 s


[]

The number of actual songplays is below its potential because not all songs were matched by the song title and the artist name.

In [19]:
%%time
%sql $con $sq.user_table_insert

105 rows affected.
CPU times: user 7.01 ms, sys: 3.83 ms, total: 10.8 ms
Wall time: 6.9 s


[]

In [20]:
%%time
%sql $con $sq.song_table_insert

14896 rows affected.
CPU times: user 11.2 ms, sys: 0 ns, total: 11.2 ms
Wall time: 2.11 s


[]

In [21]:
%%time
%sql $con $sq.artist_table_insert

10025 rows affected.
CPU times: user 11.6 ms, sys: 0 ns, total: 11.6 ms
Wall time: 2.08 s


[]

In [22]:
%%time
%sql $con $sq.time_table_insert

333 rows affected.
CPU times: user 13.8 ms, sys: 0 ns, total: 13.8 ms
Wall time: 2.2 s


[]

### Test running the DWH ETL scripts
`create_tables.py` then `etl.py`

In [23]:
%%time
import create_tables
create_tables.main()

CPU times: user 8.64 ms, sys: 1.8 ms, total: 10.4 ms
Wall time: 4.64 s


In [24]:
%%sql $con SELECT
(SELECT COUNT(*) FROM staging_events) as staging_events,
(SELECT COUNT(*) FROM staging_songs) as staging_songs,
(SELECT COUNT(*) FROM songplays) as songplays,
(SELECT COUNT(*) FROM time) as time,
(SELECT COUNT(*) FROM users) as users,
(SELECT COUNT(*) FROM songs) as songs,
(SELECT COUNT(*) FROM artists) as artists

1 rows affected.


staging_events,staging_songs,songplays,time,users,songs,artists
0,0,0,0,0,0,0


In [25]:
%%time
import etl
etl.main()

CPU times: user 7.57 ms, sys: 0 ns, total: 7.57 ms
Wall time: 2min 45s


In [26]:
%%sql $con SELECT
(SELECT COUNT(*) FROM staging_events) as staging_events,
(SELECT COUNT(*) FROM staging_songs) as staging_songs,
(SELECT COUNT(*) FROM songplays) as songplays,
(SELECT COUNT(*) FROM time) as time,
(SELECT COUNT(*) FROM users) as users,
(SELECT COUNT(*) FROM songs) as songs,
(SELECT COUNT(*) FROM artists) as artists

1 rows affected.


staging_events,staging_songs,songplays,time,users,songs,artists
8056,14896,333,333,105,14896,10025
